In [2]:
import requests
import pandas as pd
import json
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, decode

In [ ]:
# Start Spark session
spark = SparkSession.builder \
    .appName("Explore SureChemBL") \
    .getOrCreate()

# Load Data
df_mol = spark.read.parquet("../data/lake/compounds.parquet")
df_patent = spark.read.parquet("../data/lake/patent_compound_map.parquet")
df_fields = spark.read.parquet("../data/lake/fields.parquet")
df_map = spark.read.parquet("../data/lake/patent_compound_map.parquet")

In [17]:
df = df_map
df = df.join(df_mol, df.compound_id == df_mol.id, "left")
df = df.join(df_fields, df.field_id == df_fields.id, "left")
df = df.withColumn("smiles", decode(col("rdk_smiles"), "UTF-8"))

In [ ]:
df.filter(col("fieldname") == "desc").select("smiles", "mol_weight", "fieldname").show(10, truncate=False)


In [ ]:
df_fields.show(5, truncate=False)

In [ ]:
df_patent.show(5, truncate=False)

In [ ]:
df_with_smiles = df.withColumn("smiles", decode(col("rdk_smiles"), "UTF-8"))
df_with_smiles.select("id", "smiles", "mol_weight").show(5, truncate=False)

In [12]:
df['rdk_smiles']

Column<'rdk_smiles'>